In [1]:
import requests
import json
from bs4 import BeautifulSoup

In [2]:
import requests
from bs4 import BeautifulSoup
import json

url = "https://ilos.com.br/csl-e-ifr-como-converter-o-nivel-de-servico/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

title = soup.find("h1", class_="uicore-title")
meta = soup.find("div", class_="uicore-entry-meta")
article = soup.find("article")

# Autor
author = None
if meta:
    author_tag = meta.select_one('a[href*="/author/"]')
    if author_tag:
        author = author_tag.get_text(strip=True)

# Data
date_tag = meta.find("span", class_="ui-blog-date") if meta else None
date = date_tag.get_text(strip=True) if date_tag else None

# Categoria
category_tag = meta.select_one(".uicore-post-category a") if meta else None
category = category_tag.get_text(strip=True) if category_tag else None

# Conteúdo do artigo
content_div = article.find("div", class_="entry-content") if article else None
content_html = str(content_div) if content_div else None
content_text = content_div.get_text(" ", strip=True) if content_div else None

# ---------- EXTRAIR TAGS DO JSON-LD ----------
tags = []

for script in soup.find_all("script", type="application/ld+json"):
    try:
        data = json.loads(script.string)

        graph = data.get("@graph", [data])  # pode ser lista ou objeto

        for node in graph:
            if node.get("@type") == "Article":
                k = node.get("keywords")

                if isinstance(k, str):
                    k = [t.strip() for t in k.split(";")]

                tags.extend(k or [])
    except Exception:
        continue


print(json.dumps(data, ensure_ascii=False, indent=2))




thumbnail = None

for script in soup.find_all("script", type="application/ld+json"):
    try:
        data = json.loads(script.string)
        graph = data.get("@graph", [data])

        for node in graph:
            if node.get("@type") == "Article":
                thumbnail = node.get("thumbnailUrl")
    except Exception:
        continue



{
  "@context": "https://schema.org",
  "@graph": [
    {
      "@type": "Article",
      "@id": "https://ilos.com.br/csl-e-ifr-como-converter-o-nivel-de-servico/#article",
      "isPartOf": {
        "@id": "https://ilos.com.br/csl-e-ifr-como-converter-o-nivel-de-servico/"
      },
      "author": {
        "name": "Henrique Alvarenga",
        "@id": "https://ilos.com.br/#/schema/person/d09a615799493df4b74315af701fa538"
      },
      "headline": "CSL e IFR – Como converter o nível de serviço?",
      "datePublished": "2025-05-14T17:11:46+00:00",
      "dateModified": "2025-05-14T17:13:42+00:00",
      "mainEntityOfPage": {
        "@id": "https://ilos.com.br/csl-e-ifr-como-converter-o-nivel-de-servico/"
      },
      "wordCount": 1477,
      "publisher": {
        "@id": "https://ilos.com.br/#organization"
      },
      "image": {
        "@id": "https://ilos.com.br/csl-e-ifr-como-converter-o-nivel-de-servico/#primaryimage"
      },
      "thumbnailUrl": "https://ilos.com.br/wp-co

In [3]:
# Montagem do Json final com a estrutura desejada

data = {
    "title": title.get_text(strip=True) if title else None,
    "author": author,
    "date": date,
    "tags": tags,
    "thumbnail_url": thumbnail,
    "category": category,
    "content_html": str(article) if article else None,
    "source_url": url
}

with open("pagina2.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

''''
# Se quiser, ainda pode salvar o HTML para inspeção visual
with open("pagina2.html", "w", encoding="utf-8") as f:
    f.write(str(article) if article else "")'''


'\'\n# Se quiser, ainda pode salvar o HTML para inspeção visual\nwith open("pagina2.html", "w", encoding="utf-8") as f:\n    f.write(str(article) if article else "")'

In [5]:
title_text = title.get_text(strip=True) if title else ""
author_text = author or ""
date_text = date or ""
category_text = category or ""

# Monta o topo da página
header_html = f"""
<h1>{title_text}</h1>
<p style="color:#555;">
  {author_text} • {date_text} • {category_text}
</p>
<hr>
"""

# Corpo do artigo
article_html = str(article) if article else ""

# Junta tudo
pagina2_html = header_html + article_html
with open("pagina2_completa.html", "w", encoding="utf-8") as f:
    f.write(pagina2_html)